# ABS Population Growth multi-measure

## Python set-up

In [1]:
# system imports
from functools import cache
from typing import cast

# analytic imports
import pandas as pd
from pandas import PeriodIndex, Series
import readabs as ra
from readabs import metacol as mc
from mgplot import (
    line_plot_finalise,
    clear_chart_dir,
    set_chart_dir,
    multi_start,
)

# local imports
from readabs import recalibrate
from abs_structured_capture import ReqsDict, ReqsTuple, get_abs_data

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
CHART_DIR = "./CHARTS/Population/"
set_chart_dir(CHART_DIR)
clear_chart_dir()
SHOW = False
FILE_TYPE = "png"


## Download data from the ABS

In [3]:
@cache
def collect_the_data() -> dict[str, Series]:
    """Get a dictionary of data items from the ABS
    All population data is in thousands.
    All population growth data is in thousands per year.
    All population growth rate data is in percent per year.
    Returns:
    A tuple  containing adataframe of data items, and a
    dataframe of metadata."""

    # identify the data series we want to capture from the ABS
    kags = "5206001_Key_Aggregates"
    sought_after: ReqsDict = {
        # ReqsTuple: cat, table, did, stype, unit, seek_yr_growth, calc_growth, zip_file
        "Estimated Resident Population":
            ReqsTuple(
                "3101.0", 
                "310101", 
                "Estimated Resident Population (ERP) ;  Australia ;",
                "O",
                "",
                False,
                False,
                "",
            ),
            "Estimated Resident Population Annual Growth":
            ReqsTuple(
                "3101.0", 
                "310101", 
                "ERP Change Over Previous Year ;  Australia ;",
                "O",
                "000",  # needed to prevent overlap with the next one
                False,
                False,
                "",
            ),
            "Estimated Resident Population Annual Growth Rate":
            ReqsTuple(
                "3101.0", 
                "310101", 
                "Percentage ERP Change Over Previous Year ;  Australia ;",
                "O",
                "",
                False,
                False,
                "",
            ),
        "LFS Civilian Population 15+":
            ReqsTuple(
                "6202.0", 
                "6202001", 
                "Civilian population aged 15 years and over ;  Persons ;",
                "O",
                "",
                False,
                False,
                "",
            ),
        "LFS Civilian Population 15+ Annual Growth Rate":
            ReqsTuple(
                "6202.0", 
                "6202001", 
                "Civilian population aged 15 years and over ;  Persons ;",
                "O",
                "",
                False,
                True,
                "",
            ),
        "Total Monthly Arrivals":
            ReqsTuple(
                "3401.0", 
                "340101", 
                "Number of movements ;  Total Arrivals ;",
                "O",
                "",
                False,
                False,
                "",
            ),
        "Total Monthly Departures":
            ReqsTuple(
                "3401.0", 
                "340102", 
                "Number of movements ;  Total Departures ;",
                "O",
                "",
                False,
                False,
                "",
            ),
        "GDP CVM $":
            ReqsTuple(
                "5206.0", 
                kags, 
                "Gross domestic product: Chain volume measures ;",
                "O",
                "",
                False,
                False,
                "",
            ),
        "GDP per capita CVM $":
            ReqsTuple(
                "5206.0", 
                kags, 
                "GDP per capita: Chain volume measures ;",
                "O",
                "",
                False,
                False,
                "",
            ),

        }
    
    data = get_abs_data(sought_after)

    # --- calculate further statistics as needed
    data["LFS Civilian Population 15+ Annual Growth"] = data["LFS Civilian Population 15+"].diff(12)

    data["12 month rolling net total arrivals"] = (
        data["Total Monthly Arrivals"]
        - data["Total Monthly Departures"]
    ).rolling(12).sum() / 1_000

    data["Implicit population from National Accounts"] = (
        data["GDP CVM $"] / data["GDP per capita CVM $"] * 1_000
    )
    data["Implicit population (from National Accounts) growth"] = (
        # tricky: data only has active quarters so periods=4 is correct
        data["Implicit population from National Accounts"].diff(periods=4)
    )
    data["Implicit population (from National Accounts) growth rate"] = (
        # tricky: data only has active quarters so periods=4 is correct
        data["Implicit population from National Accounts"].pct_change(
            periods=4, fill_method=None  # type: ignore[arg-type]
        ) * 100
    )

    # --- convert all data to a monthly frequency
    new_dict = {}
    for kay, series in data.items():
        periodicity = cast(PeriodIndex, series.index).freqstr[0]
        if periodicity == "Q":
            series = ra.qtly_to_monthly(series)
        new_dict[kay] = series

    # done
    return new_dict


# --- test capture
if False:
    dictionary = collect_the_data()
    for key, df in dictionary.items():
        print(f"{key}:\n{df.tail()}\n")

## Plot

In [4]:
RECENT = 88  # months - touch over 7 years

In [5]:
def plot_pop() -> None:
    """plot population estimates"""

    pop_dict = collect_the_data()
    use = [
        "Estimated Resident Population",
        "LFS Civilian Population 15+",
        "Implicit population from National Accounts",
    ]
    data = {k: pop_dict[k] for k in use}
    frame = pd.DataFrame(data)
    frame, units = recalibrate(frame, "Thousands")

    multi_start(
        frame,
        function=line_plot_finalise,
        starts=(0, -RECENT),
        title="Population Estimates",
        ylabel=units,
        dropna=True,
        width=[2.5, 2, 1.5],
        style=["-", "--", "-."],
        lfooter="Australia. ",
        rfooter="ABS Cat. 3101.0, 6202.0, 5206.0",
        show=SHOW,
        file_type=FILE_TYPE,
    )


plot_pop()

In [6]:
def plot_growth() -> None:
    """plot population growth"""

    pop_dict = collect_the_data()
    use = [
        "Estimated Resident Population Annual Growth",
        "LFS Civilian Population 15+ Annual Growth",
        "Implicit population (from National Accounts) growth",
        "12 month rolling net total arrivals"
    ]
    data = {k: pop_dict[k] for k in use}
    frame = pd.DataFrame(data)
    frame, units = recalibrate(frame, "Thousands")

    multi_start(
        frame,
        function=line_plot_finalise,
        title="Population Growth",
        starts=(0, frame.index[-RECENT]),
        ylabel="Thousands per year",
        dropna=True,
        width=[3, 2.5, 2, 1.5],
        style=["-", "--", ":", "-."],
        color=["brown", "blue", "orange", "cornflowerblue"],
        y0=True,
        lfooter="Australia. ",
        rfooter="ABS Cat. 3101.0, 3401.0, 5206.0, 6202.0",
        show=SHOW,
        file_type=FILE_TYPE,
    )


plot_growth()

In [7]:
def plot_growth_rate() -> None:
    """plot population growth rate"""

    pop_dict = collect_the_data()
    use = [
        "Estimated Resident Population Annual Growth Rate",
        "LFS Civilian Population 15+ Annual Growth Rate",
        "Implicit population (from National Accounts) growth rate",
    ]
    data = {k: pop_dict[k] for k in use}
    frame = pd.DataFrame(data)

    multi_start(
        frame,
        function=line_plot_finalise,
        starts=(0, -RECENT),
        title="Population Growth Rate",
        ylabel="Annual percentage change",
        dropna=True,
        width=[2, 1.5, 1],
        style=["-", "--", "-", ":"],
        y0=True,
        lfooter="Australia. ",
        rfooter="ABS Cat. 3101.0, 5206.0, 6202.0",
        show=SHOW,
        file_type=FILE_TYPE,
    )


plot_growth_rate()

## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2026-02-19 10:20:12

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.9.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

mgplot : 0.2.18
pandas : 2.3.3
readabs: 0.1.8
typing : 3.10.0.0

Watermark: 2.6.0



In [9]:
print("Finished")

Finished
